In [ ]:
# La tabla EOP contiene clientes activos! al EOP

In [1]:
# Load kedro environment (not needed in .py)
from pathlib import Path
from kedro.framework.context import load_context
import numpy as np

# Load a context to be able to work in the notebook
#current_dir = Path.cwd()
current_dir = Path("/u01/share/cesar/aa_engine_uy/notebooks/")
proj_path = current_dir.parent
context = load_context(proj_path)
catalog = context.catalog
credentials = context.config_loader.get("credentials*","credentials*/**")
parameters = context.config_loader.get("parameters*","parameters*/**")

In [2]:
eop_connect=catalog.load("eop")

2020-12-15 14:19:33,846 - kedro.io.data_catalog - INFO - Loading data from `eop` (SQLPartitionedDataSet)...


In [3]:
eop=eop_connect.filter_by_query("select * from stg_uy_eop_customer")

/u01/miniconda3/envs/uypo37/lib/python3.7/site-packages/sqlalchemy/dialects/oracle/base.py:1381: SAWarning: Oracle version (19, 5, 0, 0, 0) is known to have a maximum identifier length of 128, rather than the historical default of 30. SQLAlchemy 1.4 will use 128 for this database; please set max_identifier_length=128 in create_engine() in order to test the application with this new length, or set to 30 in order to assure that 30 continues to be used.  In particular, pay close attention to the behavior of database migrations as dynamically generated names may change. See the section 'Max Identifier Lengths' in the SQLAlchemy Oracle dialect documentation for background.
  % ((self.server_version_info,))


In [4]:
eop.head()

,MES_ID,CUSTOMER_ID,CLI_FECHA_ACTIVACION,PRC_CODIGO,PRODUCTO,PRC_TIPO_ID,PRC_TIPO,TEC_ID,TECNOLOGIA,MOP,TENURE,COD_PAIS,DATE_EXP
0,202002,8031,2001-10-07,28,FOX +,1,Premium,30,IRD HD DVR,Invoice,220,UY,202002
1,202002,8031,2001-10-07,31,HBO MAX DIGITAL,1,Premium,30,IRD HD DVR,Invoice,220,UY,202002
2,202002,8031,2001-10-07,44,FUTBOL NACIONAL,1,Premium,30,IRD HD DVR,Invoice,220,UY,202002
3,202002,8031,2001-10-07,139,ORO,3,Basico,30,IRD HD DVR,Invoice,220,UY,202002
4,202002,8031,2001-10-07,209,Freeview Canales Locales SD,1,Premium,30,IRD HD DVR,Invoice,220,UY,202002


In [6]:
eop.groupby(["DATE_EXP"]).agg({'CUSTOMER_ID':'nunique'})

,CUSTOMER_ID
DATE_EXP,
202001,117638
202002,117386
202011,117205


In [17]:
eop.PRC_TIPO_ID.unique()

array([1, 3, 5])

In [23]:
aux=eop.loc[eop.PRC_TIPO_ID==5].groupby(["DATE_EXP","PRODUCTO"]).agg({'CUSTOMER_ID':'nunique'}).reset_index().sort_values(['CUSTOMER_ID'],ascending=False)
aux.head()

,DATE_EXP,PRODUCTO,CUSTOMER_ID
14,202011,IRD HD ONLY,97554
8,202002,IRD HD ONLY,93172
2,202001,IRD HD ONLY,92872
0,202001,IRD,39383
6,202002,IRD,38654


In [24]:
aux.pivot_table(columns=["DATE_EXP"],values="CUSTOMER_ID",index=["PRODUCTO"])

DATE_EXP,202001,202002,202011
PRODUCTO,,,
IRD,39383,38654,32294
IRD DVR 4K,568,567,575
IRD HD ONLY,92872,93172,97554
IRD High Definition,15384,15246,14242
IRD PLUS-DVR,4310,4238,3623
SWM,5,5,3


In [17]:
eop.groupby(["DATE_EXP","TEC_ID","TECNOLOGIA"]).agg({'CUSTOMER_ID':'nunique'}).reset_index().sort_values(['CUSTOMER_ID'])

,DATE_EXP,TEC_ID,TECNOLOGIA,CUSTOMER_ID
11,202002,48,4K DVR,433
5,202001,48,4K DVR,438
10,202002,40,NEXUS,2494
4,202001,40,NEXUS,2528
7,202002,20,IRD DVR,3169
1,202001,20,IRD DVR,3219
9,202002,30,IRD HD DVR,12638
3,202001,30,IRD HD DVR,12736
6,202002,10,IRD,32638
0,202001,10,IRD,32886


In [18]:
eop.groupby(["DATE_EXP","MOP"]).agg({'CUSTOMER_ID':'nunique'}).reset_index().sort_values(['CUSTOMER_ID'])

,DATE_EXP,MOP,CUSTOMER_ID
0,202001,Debito Bancario,1006
3,202002,Debito Bancario,1011
4,202002,Invoice,52937
1,202001,Invoice,53095
5,202002,Tarjeta de Credito,64447
2,202001,Tarjeta de Credito,64541


In [23]:
#activaciones
eop.loc[eop.TENURE=="0"].groupby(["DATE_EXP"]).agg({'CUSTOMER_ID':'nunique'})

,CUSTOMER_ID
DATE_EXP,
202001,1563
202002,1630
